In [1]:
%cd ../..
%pip install --use-pep517 -e .

/Users/hyan/Downloads/RL
Obtaining file:///Users/hyan/Downloads/RL
  Installing build dependencies ... one
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for BC (pyproject.toml) ... done
  Created wheel for BC: filename=BC-0.1.0-0.editable-py3-none-any.whl size=2749 sha256=45b2b25de7dcdc6cb96006ee097b11c4b38aff2b5bff2214faa737a1e9a7c715
  Stored in directory: /private/var/folders/n9/m7nc27t957s6ptyjdzxzzd7r0000gn/T/pip-ephem-wheel-cache-g6rcd46g/wheels/16/f7/da/9b141145353a924fe4828325e8436aba682c8dd74686833d9b
Successfully built BC
  Attempting uninstall: BC
    Found existing installation: BC 0.1.0
    Uninstalling BC-0.1.0:
      Successfully uninstalled BC-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gymnasium as gym

import time
import os

from infrastructure.notebook_util import *
from PG.scripts.pg_runner import run_training_loop

%load_ext autoreload
%autoreload 2

The following code might not run. I don't know what is the correct way to fix it, as it seems to be a long known issue of `moviepy`. See [https://stackoverflow.com/questions/68032884/getting-typeerror-must-be-real-number-not-nonetype-whenever-trying-to-run-wr] for example. What ends up working for me is running `pip install ffmpeg --upgrade` and restart the kernel. 

In [4]:
env = gym.make("LunarLander-v3", render_mode="rgb_array")  # Needed for rendering
env = wrap_env(env)

env.reset()
for _ in range(100):
    obs, reward, terminated, truncated, info = env.step(env.action_space.sample())  # Take random actions
    if terminated or truncated:
        result = env.reset()
        obs = result[0] if isinstance(result, tuple) else result

env.close()

print('Loading video...')
show_video(env)

/Users/hyan/anaconda3/envs/RL/lib/python3.9/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /Users/hyan/Downloads/RL/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Loading video...
video/LunarLander-v3*.mp4


In [5]:
class Args:
    def __getitem__(self, key):
        return getattr(self, key)
    def __setitem__(self, key, val):
        setattr(self, key, val)
    def __contains__(self, key):
        return hasattr(self, key)
    
    env_name:str = 'LunarLander-v3'
    folder_name:str = 'lunar'
    n_iter:int = 500
    ep_len:int = 200
    discount:float = 0.95
    gae_lambda:float = -1
    action_noise_std:float = 0
    
    use_reward_to_go:bool = True
    use_baseline:bool = True
    normalize_advantages:bool = False
    method:str = "ppo"
    exp_name:str = folder_name + "_" + method
    clip_param:float = 0.1
    
    batch_size:int = 5000
    eval_batch_size:int = 400
    
    num_agent_train_steps_per_iter:int = 1
    learning_rate:float =  0.01
    baseline_learning_rate:float = 0.01
    baseline_gradient_steps:int = 5
    
    n_layers:int = 2
    layer_size:int = 64
    
    save_params:bool = False
    no_gpu:bool = True
    which_gpu:int = 0
    seed:int = 1
    
    video_log_freq:int = 20
    scalar_log_freq:int =  1


args = Args()

args['train_batch_size'] = args['batch_size']

data_path = f"data"

if not (os.path.exists(data_path)):
    os.makedirs(data_path)

logdir = args.exp_name + '_' + args.env_name + '_' + time.strftime("%m-%d-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

In [23]:
## run training
print(args.logdir)
run_training_loop(args)

data/lunar_ppo_LunarLander-v3_02-06-2025_01-58-35
###########################
logging outputs to  data/lunar_ppo_LunarLander-v3_02-06-2025_01-58-35
###########################
Using CPU

********** Iteration 0 ************

Eval_AverageReturn : -146.2856903076172
Eval_StdReturn : 58.63264083862305
Eval_MaxReturn : -88.99750518798828
Eval_MinReturn : -256.6947021484375
Eval_AverageEpLen : 81.4
Train_AverageReturn : -175.9008026123047
Train_StdReturn : 99.68949127197266
Train_MaxReturn : -11.600448608398438
Train_MinReturn : -436.9567565917969
Train_AverageEpLen : 90.67857142857143
Actor Loss : 35.28310775756836
Baseline Loss : 2199.483154296875
Train_EnvstepsSoFar : 5078
TimeSinceStart : 0.608253002166748
Initial_DataCollection_AverageReturn : -175.9008026123047
Done logging...



(86, 3, 250, 250)
(116, 3, 250, 250)

********** Iteration 1 ************

Eval_AverageReturn : -123.51644134521484
Eval_StdReturn : 70.61949157714844
Eval_MaxReturn : 5.123497009277344
Eval_MinReturn : -201.2

In [6]:
%load_ext tensorboard
%tensorboard --logdir ./data